In [1]:
data_directory = '../data/centralities/'
class_system = 'USPC'
controls_directory = data_directory+'controls/%s/'%class_system

In [2]:
n_controls = 1000

In [3]:
import pandas as pd
from pylab import *

In [4]:
import gc
from time import time

In [5]:
empirical = pd.read_hdf(data_directory+'empirical.h5', 'df')

In [6]:
def running_stats(empirical,
                  df_name,
                  file_name,
                  controls_directory=controls_directory,
                  n_controls=n_controls,
                 ):
    M = None
    all_max = None
    all_min = None
    t = time()
    for randomization_id in range(n_controls):

        if not randomization_id%10:
            print(randomization_id)
            print("%.0f seconds"%(time()-t))
            t = time()
        
        f = '%s_%i.h5'%(file_name, randomization_id)
        try:
            x = pd.read_hdf(controls_directory+f, df_name)
        except:
            print("Data not loading for %s. Continuing."%f)
            continue
            
        if M is None:
            M = x
            S = 0
#             all_max = M
#             all_min = M
            p = 0
            k = 0
            continue
        k += 1.0
        M_previous = M
#         M = M_previous.add( x.subtract(M_previous)/k )
#         S = ( x.subtract(M_previous).multiply( x.subtract(M) ) ).add(S)
#         print(x.dtypes)
#         print(x.shape)
#         empirical.iloc[:,0]>x.iloc[:,0]
        M = M_previous+((x-M_previous)/k)
        S += (x-M_previous)*(x-M)
        p += (empirical>x).astype('int')
#         all_max = maximum(all_max, x)
#         all_min = minimum(all_min, x)
        gc.collect()  
    standard_deviation = sqrt(S/(k-1))
    z_scores = (empirical-M)/standard_deviation
    print(k)
    return M, S, p/k, z_scores

In [7]:
M, standard_deviation, empirical_percentile, empirical_z_scores = running_stats(empirical, 'df', 'synthetic_control', controls_directory)

0
0 seconds
10
34 seconds
20
34 seconds
30
34 seconds
40
34 seconds
50
34 seconds
60
34 seconds
70
34 seconds
80
34 seconds
90
34 seconds
100
34 seconds
110
34 seconds
120
34 seconds
130
34 seconds
140
34 seconds
150
34 seconds
160
34 seconds
170
35 seconds
180
38 seconds
190
38 seconds
200
37 seconds
210
38 seconds
220
38 seconds
230
38 seconds
240
38 seconds
250
37 seconds
260
37 seconds
270
37 seconds
280
36 seconds
290
36 seconds
300
37 seconds
310
40 seconds
320
36 seconds
330
32 seconds
340
33 seconds
350
33 seconds
360
32 seconds
370
33 seconds
380
34 seconds
390
33 seconds
400
33 seconds
410
34 seconds
420
32 seconds
430
34 seconds
440
34 seconds
450
32 seconds
460
33 seconds
470
34 seconds
480
32 seconds
490
35 seconds
500
36 seconds
510
37 seconds
520
36 seconds
530
37 seconds
540
36 seconds
550
37 seconds
560
37 seconds
570
36 seconds
580
37 seconds
590
36 seconds
600
36 seconds
610
36 seconds
620
37 seconds
630
38 seconds
640
38 seconds
650
39 seconds
660
41 seconds
670
41 

In [8]:
for df in [M, standard_deviation, empirical_percentile, empirical_z_scores]:
    df['patent_number'] = empirical['patent_number']
    df['filing_year'] = empirical['filing_year']

In [9]:
store = pd.HDFStore(data_directory+'summary_statistics.h5',
                        mode='a', table=True)
store.put('/randomized_mean_%s'%(class_system), M, 'table', append=False)
store.put('/randomized_std_%s'%(class_system), standard_deviation, 'table', append=False)

store.put('/empirical_percentile_%s'%(class_system), empirical_percentile, 'table', append=False)
# store.put('/randomized_min_%s'%(class_system), all_min, 'table', append=False)

z_scores = empirical_z_scores

z_scores.values[where(z_scores==inf)]=nan 
z_scores.values[where(z_scores==-inf)]=nan 
#All the cases where the z-scores are inf is where the 1,000 randomized controls said there should be 0 deviation, BUT
#the empirical case was different anyway. In each of these cases, the empirical case was JUST slightly off. Sometimes
#a floating point error, and sometimes off by 1 (the minimal amount for citation counts). We shall treat this as not actually
#deviating, and so it becomes 0/0, which is equal to nan.

store.put('/empirical_z_scores_%s'%(class_system), z_scores, 'table', append=False)

store.put('/empirical', empirical, 'table', append=False)

store.close()

In [10]:
cp /home/jeffrey_alstott/technoinnovation/patent_centralities/data/centralities/summary_statistics.h5 /home/jeffrey_alstott/Dropbox/TechnoInnovation/